# Import

In [1]:
import astropy.units as u
from astropy.coordinates.sky_coordinate import SkyCoord
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt
import os
import copy

from kesacco import clustpipe

NameError: name 'u' is not defined

# Parameters

### Define the simulation

In [ ]:
cpipe = clustpipe.ClusterPipe(silent=False, output_dir='/Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev')

### Define the cluster object

In [ ]:
# Set the cluster basic properties
cpipe.cluster.name     = 'Perseus'
cpipe.cluster.redshift = 0.017284
cpipe.cluster.M500     = 6.2e14*u.solMass # Used for getting R500 and the pressure profile
cpipe.cluster.coord    = SkyCoord("3h19m47.2s +41d30m47s", frame='icrs')

# Truncate the cluster at 1.5 deg to allow small map field of view (for faster code) without loss of flux
cpipe.cluster.theta_truncation = 1.5*u.deg

# The target gas density [Churazov et al. 2003]
cpipe.cluster.density_gas_model = {'name':'doublebeta', 'beta1':1.8, 'r_c1':80*u.kpc, 'n_01':3.9e-2*u.cm**-3,
                                   'beta2':0.87, 'r_c2':280*u.kpc, 'n_02':4.05e-3*u.cm**-3}

# The thermal profile (assuming Planck 2013 UPP)
cpipe.cluster.set_pressure_gas_gNFW_param('P13UPP')

# CR physics
cpipe.cluster.spectrum_crp_model = {'name':'PowerLaw', 'Index':2.2}
cpipe.cluster.set_density_crp_isobaric_scal_param(scal=1.0)
cpipe.cluster.X_crp_E = {'X':1.0, 'R_norm':cpipe.cluster.R500}

# Sampling
cpipe.cluster.map_reso = 0.01*u.deg      # Ideally should be few times smaller than the PSF
cpipe.cluster.Npt_per_decade_integ = 30

# Get information about the state of the cluster model
cpipe.cluster.print_param()

### Define the point source object

In [ ]:
name='NGC1275'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h19m48.16s +41d30m42s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h19m48.16s +41d30m42s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':2.1e-11/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-3.6, 'free':True},
                   'PivotEnergy':{'value':0.2*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral) # Add the source to the model

In [ ]:
name='IC310'
spatial={'type':'PointSource',
         'param':{'RA': {'value':SkyCoord("3h16m42.98s +41d19m30s").ra.to('deg'),  'free':False},
                  'DEC':{'value':SkyCoord("3h16m42.98s +41d19m30s").dec.to('deg'), 'free':False}}}
spectral={'type':'PowerLaw',
          'param':{'Prefactor':{'value':4.3e-12/u.cm**2/u.TeV/u.s, 'free':True},
                   'Index':{'value':-1.95, 'free':True},
                   'PivotEnergy':{'value':1.0*u.TeV, 'free':False}}}

cpipe.compact_source.add_source(name, spatial, spectral)# Add the source to the model

In [ ]:
# Show the status of the compact sources in the sky model
cpipe.compact_source.print_source()

### Define the observations

In [ ]:
cpipe.obs_setup.add_obs(obsid='001', name='Perseus_Ptg1', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+0, cpipe.cluster.coord.dec.value+1, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-01T00:00:00.0', tmax='2020-01-01T00:30:00.0', deadc=0.95)

# One pointing offset +sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='002', name='Perseus_Ptg2', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value+np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg, 
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-02T00:00:00.0', tmax='2020-01-02T00:30:00.0', deadc=0.95)

# One pointing offset -sqrt(3)/2 -0.5
cpipe.obs_setup.add_obs(obsid='003', name='Perseus_Ptg3', 
                        coord=SkyCoord(cpipe.cluster.coord.ra.value-np.sqrt(3)/2, cpipe.cluster.coord.dec.value-0.5, unit='deg'),
                        rad=5*u.deg,
                        emin=0.05*u.TeV, emax=100*u.TeV,
                        caldb='prod3b-v2', irf='North_z20_S_5h',
                        tmin='2020-01-03T00:00:00.0', tmax='2020-01-03T00:30:00.0', deadc=0.95)

# Print info
cpipe.obs_setup.print_obs()

# Run the simulation

In [8]:
#cpipe.run_sim_obs()
cpipe.run_sim_quicklook(ShowEvent=True,
                        ShowObsDef=True,
                        ShowSkyModel=True,
                        bkgsubtract='RING',
                        smoothing_FWHM=0.1*u.deg)

----- ObsID to be observed: ['001', '002', '003']
------- Simulation log -------
=== GApplication ===
 Name ......................: ctobssim
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_ObsDef.xml
 inmodel ...................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Model_Unstack.xml
 caldb .....................: prod2
 irf .......................: South_0.5h
 edisp .....................: no
 outevents .................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/TmpEvents
 startindex ................: 1
 seed ......................: 499297
 ra ........................: 83.63
 dec .......................: 22.51
 rad .......................: 5.0
 tmin ......................: 2020-01-01T00:00:00
 tmax ......................: 2020-01-01T00:30:00
 mjdref ....................: 51544.5
 emin ....................

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:111: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  c    = sub.imshow(image, extent=[emin,emax,tmin,tmax], aspect=0.5, vmin=1e6, vmax=5e10, norm=LogNorm())
/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting_irf.py:474: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  vmin=1e-12, vmax=1, norm=LogNorm())


!!! WARNING: XSPEC_table.txt not generated, skip Xray flux and Sx
!!! WARNING: XSPEC_table.txt not generated, skip Xray spectrum
!!! WARNING: XSPEC_table.txt not generated, skip PLOT_MAP_X
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events001.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Skymap001.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: RING
 

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:385: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 1/3 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events002.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Skymap002.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: RING
 roiradius .................: 0.3
 inradius ..................: 0.8
 outradius .................: 1
 iterations ................: 3
 threshold .................: 3
 

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:385: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 2/3 done
=== GApplication ===
 Name ......................: ctskymap
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Events003.fits
 caldb .....................: prod3b-v2
 irf .......................: North_z20_S_5h
 inmap .....................: NONE
 outmap ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Sim_Skymap003.fits
 emin ......................: 0.05
 emax ......................: 100
 usepnt ....................: no
 nxpix .....................: 501
 nypix .....................: 501
 binsz .....................: 0.01
 coordsys ..................: CEL
 proj ......................: TAN
 xref ......................: 49.9466666666667
 yref ......................: 41.5130555555556
 bkgsubtract ...............: RING
 roiradius .................: 0.3
 inradius ..................: 0.8
 outradius .................: 1
 iterations ................: 3
 threshold .................: 3
 

/Users/adam/Project/CTA/Phys/Software/kesacco/Tools/plotting.py:385: MatplotlibDeprecationWarning: default base will change from np.e to 10 in 3.4.  To suppress this warning specify the base keyword argument.
  plt.imshow(image, origin='lower', cmap=cmap, norm=SymLogNorm(1), vmin=vmin, vmax=vmax)
/Users/adam/Project/anaconda3/envs/ctaphys/lib/python3.7/site-packages/astropy/visualization/wcsaxes/core.py:211: MatplotlibDeprecationWarning: Passing parameters norm and vmin/vmax simultaneously is deprecated since 3.3 and will become an error two minor releases later. Please pass vmin/vmax directly to the norm when creating it.
  return super().imshow(X, *args, origin=origin, **kwargs)


----- Quicklook 3/3 done


# Run the analysis

In [7]:
#----- Analysis parameters
cpipe.method_binned = True    # Do a binned analysis
cpipe.method_stack  = True    # Stack the event from different observations in a single analysis?
cpipe.method_ana    = 'ONOFF' # 3D or ONOFF analysis
cpipe.spec_enumbins = 10
cpipe.spec_emin     = 50*u.GeV
cpipe.spec_emax     = 10*u.TeV
cpipe.spec_edisp    = False

# Force the use of the user defined map grid
cpipe.map_UsePtgRef     = False
# Define the map used for the binned analysis
cpipe.map_reso          = 0.05*u.deg # Can be increaded if the code is too slow
cpipe.map_fov           = 3*u.deg    # Can also be reduced (but should increase bkg-cluster degeneracy)
cpipe.map_coord         = copy.deepcopy(cpipe.cluster.coord)
# Define the map used for the template
cpipe.cluster.map_fov   = 2.1*cpipe.cluster.theta_truncation
cpipe.cluster.map_coord = copy.deepcopy(cpipe.cluster.coord)

In [8]:
bid = cpipe.run_ana_dataprep(obsID=None, frac_src_on_reg=0.8, use_model_bkg=True)
Ldat = cpipe.run_ana_likelihood()


 Starting the data preparation

         This should be used when simulating the data.  
----- ObsID to be analysed: ['001', '002']

=== GApplication ===
 Name ......................: ctselect
 Version ...................: 1.7.3
 inobs .....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Events.xml
 outobs ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_EventsSelected.xml
 prefix ....................: /Users/adam/Project/CTA/Phys/Outputs/KESACCO_Dev/Ana_Selected
 usepnt ....................: no
 ra ........................: 49.9466666666667
 dec .......................: 41.5130555555556
 rad .......................: 2.12132034355964
 forcesel ..................: yes
 tmin ......................: NONE
 tmax ......................: NONE
 emin ......................: 0.05
 emax ......................: 10
 phase .....................: NONE
 expr ......................: 
 usethres ..................: NONE
 publish ...................: no
 c

=== GOptimizerLM ===
 Optimized function value ..: -97246.398
 Absolute precision ........: 0.005
 Acceptable value decrease .: 2
 Optimization status .......: converged
 Number of parameters ......: 11
 Number of free parameters .: 5
 Number of iterations ......: 3
 Lambda ....................: 1e-06
=== GObservations ===
 Number of observations ....: 2
 Number of models ..........: 3
 Number of observed events .: 4547
 Number of predicted events : 3298.74464526829
=== GModels ===
 Number of models ..........: 3
 Number of parameters ......: 11
=== GModelSky ===
 Name ......................: Perseus
 Instruments ...............: all
 Test Statistic ............: 1162.20567280648
 Observation identifiers ...: all
 Model type ................: DiffuseSource
 Model components ..........: "DiffuseMap" * "FileFunction" * "Constant"
 Number of parameters ......: 3
 Number of spatial par's ...: 1
  Normalization ............: 1 [0.001,1000]  (fixed,scale=1,gradient)
 Number of spectral par's